# NFL Combine Drill Distributions

Beeswarm plots for all six combine drills — every recorded measurement from 2007 to 2026, colored by position group.

Python port of the original R scripts (`ggbeeswarm::geom_quasirandom` → `seaborn.stripplot`).

In [ ]:
import os

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from matplotlib import font_manager as fm

# --- Brand setup ---
BG = "#DDEBEC"
TEXT = "#333333"
CAPTION = "Ray Carpenter | TheSpade.Substack.com | @csv_enjoyer | data: Various sources"

# Okabe-Ito palette (colorblind-safe)
POS_COLORS = {
    "DB": "#0072B2",
    "WR": "#E69F00",
    "RB": "#009E73",
    "LB": "#D55E00",
    "TE": "#CC79A7",
    "FB": "#F0E442",
    "QB": "#56B4E9",
    "P": "#000000",
    "DL": "#E83562",
    "LS": "#7A5195",
    "OL": "#1AFF1A",
}
POS_ORDER = list(POS_COLORS.keys())

OUTPUT_DIR = os.path.join("..", "output", "beeswarm")
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# Load the coalesced combine + pro day dataset
df = pd.read_parquet(os.path.join("..", "data", "combine_pro_day.parquet"))

# Collapse EDGE/DT into DL
df["POS_GP"] = df["POS_GP"].replace({"EDGE": "DL", "DT": "DL"})
df = df[df["POS_GP"].isin(POS_ORDER)].copy()
df["POS_GP"] = pd.Categorical(df["POS_GP"], categories=POS_ORDER, ordered=True)

print(f"Loaded {len(df):,} rows ({df['Year'].min()}–{df['Year'].max()})")

In [ ]:
def beeswarm(col, title, xlabel, lo, hi, step, filename):
    """One beeswarm strip plot for a single combine drill."""
    subset = df.dropna(subset=[col]).copy()
    subset = subset[(subset[col] >= lo) & (subset[col] <= hi)]
    n = len(subset)

    fig, ax = plt.subplots(figsize=(16, 9))
    fig.patch.set_facecolor(BG)
    ax.set_facecolor(BG)

    sns.stripplot(
        data=subset, x=col, y=[""]*n,
        hue="POS_GP", hue_order=POS_ORDER, palette=POS_COLORS,
        jitter=0.4, size=2, alpha=0.6, ax=ax, legend=True,
    )

    ax.set_title(title, fontsize=28, fontweight="bold", color=TEXT, loc="left", pad=20)
    ax.text(
        0.0, 1.04, f"Every recorded measurement from 2007–2026 (n = {n:,})",
        transform=ax.transAxes, fontsize=16, color="#666666",
    )
    ax.set_xlabel(xlabel, fontsize=16, color=TEXT)
    ax.set_ylabel("")
    ax.set_yticks([])
    ax.tick_params(axis="x", labelsize=14)
    import numpy as np
    ax.set_xticks(np.arange(lo, hi + step, step))

    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_visible(False)

    legend = ax.get_legend()
    legend.set_title(None)
    legend.set_frame_on(False)
    for text in legend.get_texts():
        text.set_fontsize(12)
        text.set_fontweight("bold")

    fig.text(0.01, 0.01, CAPTION, fontsize=11, color="#888888")

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    path = os.path.join(OUTPUT_DIR, filename)
    fig.savefig(path, dpi=300, facecolor=BG, bbox_inches="tight")
    print(f"Saved {path}")
    plt.show()
    plt.close(fig)

In [ ]:
beeswarm(
    col="40 Yard",
    title="NFL Combine & Pro Day 40-Yard Dash Times by Position",
    xlabel="40-Yard Dash (seconds)",
    lo=4.2, hi=5.5, step=0.2,
    filename="forty_spread.png",
)

In [ ]:
beeswarm(
    col="Vert Leap (in)",
    title="NFL Combine & Pro Day Vertical Leap by Position",
    xlabel="Vertical Leap (inches)",
    lo=20, hi=46, step=2,
    filename="vert_leap_spread.png",
)

In [ ]:
beeswarm(
    col="Broad Jump (in)",
    title="NFL Combine & Pro Day Broad Jump by Position",
    xlabel="Broad Jump (inches)",
    lo=84, hi=142, step=4,
    filename="broad_jump_spread.png",
)

In [ ]:
beeswarm(
    col="3Cone",
    title="NFL Combine & Pro Day 3-Cone Drill by Position",
    xlabel="3-Cone Drill (seconds)",
    lo=6.2, hi=8.2, step=0.2,
    filename="three_cone_spread.png",
)

In [ ]:
beeswarm(
    col="Shuttle",
    title="NFL Combine & Pro Day 20-Yard Shuttle by Position",
    xlabel="20-Yard Shuttle (seconds)",
    lo=3.8, hi=5.0, step=0.2,
    filename="shuttle_spread.png",
)

In [ ]:
beeswarm(
    col="Bench Press",
    title="NFL Combine & Pro Day Bench Press by Position",
    xlabel="Bench Press (reps at 225 lbs)",
    lo=5, hi=50, step=5,
    filename="bench_press_spread.png",
)